In [2]:
# Run a mock of fractal-demos/examples/01, with an additional
# illumination correction task which does *not* overwrite input
# images.

In [8]:
# Preliminary imports and auxiliary-function definition

from pathlib import Path
from devtools import debug
import tempfile

from fractal_server.app.runner.v2.models import Dataset, WorkflowTask
from fractal_server.app.runner.v2.runner import execute_tasks_v2
from fractal_server.images import SingleImage

import sys
sys.path.append("../tests/v2/04_runner")
from fractal_tasks_core_mock import TASK_LIST

from concurrent.futures import ThreadPoolExecutor

executor = ThreadPoolExecutor()



def image_data_exist_on_disk(image_list: list[SingleImage]):
    """
    Given an image list, check whether mock data were written to disk.
    """
    all_images_have_data = True
    for image in image_list:
        if (Path(image.path) / "data").exists():
            print(f"[image_data_exist_on_disk] {image.path} contains data")
        else:
            print(f"[image_data_exist_on_disk] {image.path} does *not* contain data")
            all_images_have_data = False
        
    return all_images_have_data

In [9]:
# Create temporary directory for mocked zarrs
tmp_path = Path(tempfile.mkdtemp())
zarr_dir = (tmp_path / "zarr_dir").as_posix().rstrip("/")
print(f"{zarr_dir=}")

zarr_dir='/tmp/tmpezqwkd6j/zarr_dir'


In [10]:
# Run create-ome-zarr (starting from empty Dataset)
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["create_ome_zarr_compound"],
            args_non_parallel=dict(image_dir="/tmp/input_images"),
        )
    ],
    dataset=Dataset(zarr_dir=zarr_dir),
    executor=executor,
)

# Images were added, but data do not exist on-disk yet
assert image_data_exist_on_disk(dataset.images)

# Print dataset metadata
debug(dataset)

[create_ome_zarr] START
[create_ome_zarr] image_dir='/tmp/input_images'
[create_ome_zarr] zarr_dir='/tmp/tmpezqwkd6j/zarr_dir'
[create_ome_zarr] zarr_path='/tmp/tmpezqwkd6j/zarr_dir/my_plate.zarr'
[create_ome_zarr] END
[yokogawa_to_zarr] START
[yokogawa_to_zarr] path='/tmp/tmpezqwkd6j/zarr_dir/my_plate.zarr/A/01/0'
[yokogawa_to_zarr] raw_path='/tmp/input_images/A_01_0.tif'
[yokogawa_to_zarr] START
[yokogawa_to_zarr] path='/tmp/tmpezqwkd6j/zarr_dir/my_plate.zarr/A/02/0'
[yokogawa_to_zarr] raw_path='/tmp/input_images/A_02_0.tif'
[yokogawa_to_zarr] END
[yokogawa_to_zarr] END
/home/marco/exact/fractal-server/fractal_server/app/runner/v2/runner_functions.py:105 merge_outputs
    'MERGE OUTPUTS' (str) len=13
/home/marco/exact/fractal-server/fractal_server/app/runner/v2/runner_functions.py:106 merge_outputs
    task_outputs: [
        TaskOutput(
            added_images=[
                SingleImage(
                    path='/tmp/tmpezqwkd6j/zarr_dir/my_plate.zarr/A/01/0',
                 

Dataset(id=None, history=[None], zarr_dir='/tmp/tmpezqwkd6j/zarr_dir', images=[SingleImage(path='/tmp/tmpezqwkd6j/zarr_dir/my_plate.zarr/A/01/0', attributes={'well': 'A01', 'plate': 'my_plate.zarr', 'data_dimensionality': 3}), SingleImage(path='/tmp/tmpezqwkd6j/zarr_dir/my_plate.zarr/A/02/0', attributes={'well': 'A02', 'plate': 'my_plate.zarr', 'data_dimensionality': 3})], filters={'plate': 'my_plate.zarr', 'data_dimensionality': 3})

In [11]:
# Run illumination correction, with overwrite_input=True
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["illumination_correction"],
            args=dict(overwrite_input=False),
        )
    ],
    dataset=dataset,
    executor=executor,
)

# Image data still exist (and they actually were modified)
assert image_data_exist_on_disk(dataset.images)

# Print dataset metadata
debug(dataset);

[illumination_correction] START
[illumination_correction] path='/tmp/tmpezqwkd6j/zarr_dir/my_plate.zarr/A/01/0'
[illumination_correction] overwrite_input=False
[illumination_correction] START
[illumination_correction] path='/tmp/tmpezqwkd6j/zarr_dir/my_plate.zarr/A/02/0'
[illumination_correction] overwrite_input=False
[illumination_correction] new_path='/tmp/tmpezqwkd6j/zarr_dir/my_plate.zarr/A/02/0_corr'
[illumination_correction] new_path='/tmp/tmpezqwkd6j/zarr_dir/my_plate.zarr/A/01/0_corr'
[illumination_correction] END
[illumination_correction] END
[new_ome_zarr] START
[new_ome_zarr] paths=['/tmp/tmpezqwkd6j/zarr_dir/my_plate.zarr/A/01/0_corr', '/tmp/tmpezqwkd6j/zarr_dir/my_plate.zarr/A/02/0_corr']
[new_ome_zarr] Identified old_plate='my_plate.zarr'
[new_ome_zarr] new_plate='my_plate_mip.zarr'
[new_ome_zarr] new_zarr_path='/tmp/tmpezqwkd6j/zarr_dir/my_plate_mip.zarr'
[new_ome_zarr] END
[maximum_intensity_projection] START
[maximum_intensity_projection] path='/tmp/tmpezqwkd6j/zarr_di

/home/marco/exact/fractal-server/fractal_server/app/runner/v2/runner_functions.py:105 merge_outputs
    'MERGE OUTPUTS' (str) len=13
/home/marco/exact/fractal-server/fractal_server/app/runner/v2/runner_functions.py:106 merge_outputs
    task_outputs: [
        TaskOutput(
            added_images=[
                SingleImage(
                    path='/tmp/tmpezqwkd6j/zarr_dir/my_plate.zarr/A/01/0_corr',
                    attributes={},
                ),
            ],
            edited_images=None,
            removed_images=None,
            new_filters=None,
        ),
        TaskOutput(
            added_images=[
                SingleImage(
                    path='/tmp/tmpezqwkd6j/zarr_dir/my_plate.zarr/A/02/0_corr',
                    attributes={},
                ),
            ],
            edited_images=None,
            removed_images=None,
            new_filters=None,
        ),
    ] (list) len=2
/home/marco/exact/fractal-server/fractal_server/app/runner/v2/runn

In [12]:
# Run new-ome-zarr, as an init task for MIP
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["MIP_compound"],
            args_non_parallel=dict(suffix="mip"),
        )
    ],
    dataset=dataset,
    executor=executor,
)

# 3D image data exist on disk
images_3D = [
        image
        for image in dataset.images
        if image.attributes["data_dimensionality"] == 3
    ]
assert image_data_exist_on_disk(images_3D)

# 2D image data do not exist on disk
images_2D = [
        image
        for image in dataset.images
        if image.attributes["data_dimensionality"] == 2
    ]
assert image_data_exist_on_disk(images_2D)

# Print dataset metadata
debug(dataset);

/home/marco/exact/fractal-server/fractal_server/app/runner/v2/runner_functions.py:105 merge_outputs
    'MERGE OUTPUTS' (str) len=13
/home/marco/exact/fractal-server/fractal_server/app/runner/v2/runner_functions.py:106 merge_outputs
    task_outputs: [
        TaskOutput(
            added_images=[
                SingleImage(
                    path='/tmp/tmpezqwkd6j/zarr_dir/my_plate_mip.zarr/A/01/0_corr',
                    attributes={},
                ),
            ],
            edited_images=None,
            removed_images=None,
            new_filters={
                'plate': 'my_plate_mip.zarr',
                'data_dimensionality': 2,
            },
        ),
        TaskOutput(
            added_images=[
                SingleImage(
                    path='/tmp/tmpezqwkd6j/zarr_dir/my_plate_mip.zarr/A/02/0_corr',
                    attributes={},
                ),
            ],
            edited_images=None,
            removed_images=None,
            new_fil

In [13]:
# Run cellpose on current images
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["cellpose_segmentation"],
            args_parallel=dict(),
        )
    ],
    dataset=dataset,
    executor=executor,
)

# All image data (3D and 2D) now exists on disk
assert image_data_exist_on_disk(dataset.images)

# Print dataset metadata
debug(dataset);

/home/marco/exact/fractal-server/fractal_server/app/runner/v2/runner_functions.py:105 merge_outputs
    'MERGE OUTPUTS' (str) len=13
/home/marco/exact/fractal-server/fractal_server/app/runner/v2/runner_functions.py:106 merge_outputs
    task_outputs: [
        TaskOutput(
            added_images=None,
            edited_images=None,
            removed_images=None,
            new_filters=None,
        ),
        TaskOutput(
            added_images=None,
            edited_images=None,
            removed_images=None,
            new_filters=None,
        ),
    ] (list) len=2
/home/marco/exact/fractal-server/fractal_server/app/runner/v2/runner_functions.py:107 merge_outputs
    new_old_image_mapping: {} (dict) len=0
/home/marco/exact/fractal-server/fractal_server/app/runner/v2/runner_functions.py:108 merge_outputs
    old_dataset_images: [
        SingleImage(
            path='/tmp/tmpezqwkd6j/zarr_dir/my_plate_mip.zarr/A/01/0_corr',
            attributes={
                'well':

In [10]:
executor.shutdown()